In [2]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pandas as pd
import datetime as dt

## Reflect Tables into SQLALchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [6]:
# reflect an existing database into a new model

# reflect the tables

In [7]:
# View all of the classes that automap found


In [8]:
# Save references to each table

In [9]:
# Create our session (link) from Python to the DB

## Bonus Challenge Assignment: Temperature Analysis II

In [10]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, maximum, and average temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# For example
print(calc_temps('2012-02-28', '2012-03-05'))

NameError: name 'session' is not defined

In [ ]:
# Use the function `calc_temps` to calculate the tmin, tavg, and tmax 
# for a year in the data set

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for bar height (y value)
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)

### Daily Rainfall Average

In [2]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's 
# matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation
sel = [Measurement.station, 
       func.sum(Measurement.prcp),
       Station.name,
       Station.latitude,
       Station.longitude,
       Station.elevation
      ]
station_totals = session.query(*sel).\
        filter(Measurement.station == Station.station, func.strftime(Measurement.date) >= start_date, func.strftime(Measurement.date) <= end_date).\
        group_by(Measurement.station).\
        order_by(func.sum(Measurement.prcp).desc()).all()

station_totals

NameError: name 'Measurement' is not defined

### Daily Temperature Normals

In [ ]:
# Use this function to calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()

# For example
daily_normals("01-01")

In [3]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip
start_date = '2017-08-01'
end_date = '2017-08-07'

# Use the start and end date to create a range of dates
form_start = dt.datetime.strptime(start_date,'%Y-%m-%d')
form_end = dt.datetime.strptime(end_date,'%Y-%m-%d')
trip_length = form_end - form_start

# Strip off the year and build a list of %m-%d stringsfor each date
normal_dates = []
for i in range(trip_length.days + 1):
    current_day = form_start + dt.timedelta(days=i)
    month_day = dt.datetime.strftime(current_day,'%m-%d')
    #normals.append(month_day)
    normal_dates.append(month_day)

# Loop through the list of %m-%d strings and calculate the normals for each date
normals = []
for day in normal_dates:
    normals.append(daily_normals(day)[0])

NameError: name 'dt' is not defined

In [4]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index
normals_df = pd.DataFrame(normals,columns=["Minimum Temperature", "Average Temperature", "Maximum Temperature"], index=[normal_dates])
normals_df.index = normals_df.index.get_level_values(0)
normals_df.head(100)

NameError: name 'pd' is not defined

In [ ]:
# Plot the daily normals as an area plot with `stacked=False`

ax = normals_df.plot(kind='area', stacked=False, alpha=.2, rot=45, title = "Temperature Normals for Trip Dates", xlabel='Date', ylabel = "Temperature (F)", figsize =(8,6))
plt.tight_layout()
plt.show()
ax.figure.savefig('TempNormalsforTrip.png', bbox_inches='tight')

## Close Session